# Modlling for engagement 

now we have the features and the lables, we are ready for modelling 
- Classification vs regression
- 9 vs 5 vs 3 scales lables 
- 2.5 vs 5 s window (as is features, average featuers and concatenate features)
- Two-stream Fusion on RGB + Flow 

This should be done over:
- different network artchictures
- cross different familys cross-validation 

Later on:
- Handcrafted features 

### imports and vars

In [1]:
import sys
import numpy as np
import os
import pandas as pd
import itertools
import pickle

import matplotlib.pyplot as plt
from collections import Counter
from sklearn.utils import shuffle
import resreg

lables_path = './labels/'
features_path = './features/'
results_path = './modelling_results/'

### Keras modeling class 

In [2]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

import ConfigSpace as CS
import ConfigSpace.hyperparameters as CSH

from hpbandster.core.worker import Worker
from hpbandster.optimizers import BOHB as BOHB
import hpbandster.core.nameserver as hpns
import hpbandster.core.result as hpres

import tensorflow_addons as tfa

import logging
logging.basicConfig(level=logging.ERROR)

class KerasWorker(Worker):
    def __init__(self, input_shape, output_shape, problemType,
                 x_train, y_train, x_validation, y_validation,
                 x_test, y_test, shared_directory, **kwargs):
            super().__init__(**kwargs)
            self.input_shape = (input_shape, )
            self.num_classes = output_shape
            self.batch_size = 64
            self.save_dic = shared_directory
            
            self.problemType = problemType

            self.x_train, self.y_train = x_train, y_train
            self.x_validation, self.y_validation = x_validation, y_validation
            self.x_test, self.y_test = x_test, y_test

    def compute(self, config, budget, working_directory, *args, **kwargs):
            model = Sequential()
            model.add(Dense(units=config['start_neurons_units'],
                            # activation=config['start_neurons_activation'],
                            activation='relu',
                            input_shape=self.input_shape))


            if config['num_dense_layers'] > 1:
                model.add(Dense(units=config['dense1_units'],
                                # activation=config['dense1_activation'],
                                activation='relu',
                                input_shape=self.input_shape))
                model.add(Dropout(config['dropout1_rate']))

            if config['num_dense_layers'] > 2:
                model.add(Dense(units=config['dense2_units'],
                                # activation=config['dense2_activation'],
                                activation='relu',
                                input_shape=self.input_shape))
                model.add(Dropout(config['dropout2_rate']))

            model.add(Dense(self.num_classes, activation='softmax'))


            if config['optimizer'] == 'Adam':
                    optimizer = tf.keras.optimizers.Adam(lr=config['lr'])
            else:
                    optimizer = tf.keras.optimizers.SGD(lr=config['lr'], momentum=config['sgd_momentum'])
            
            loss_fn = tf.keras.losses.MeanSquaredError(reduction="auto", name="mse")
            
            METRICS = [
                tf.keras.losses.MeanAbsoluteError(reduction="auto", name="mae"),
                tf.keras.losses.MeanAbsolutePercentageError(reduction="auto", name="maperc_e"),
                tf.keras.losses.MeanSquaredLogarithmicError(reduction="auto", name="mslog_e"),
                tf.keras.losses.CosineSimilarity(reduction="auto", name="CosineSimilarity"),
                tf.keras.losses.Huber(reduction="auto", name="Huber"),
                tf.keras.losses.LogCosh(reduction="auto", name="LogCosh"),
            ]

            #METRICS.append(tfa.metrics.MatthewsCorrelationCoefficient(num_classes=self.num_classes))
            
            the_columns = ['loss','mae','maperc_e','mslog_e','CosineSimilarity','Huber','LogCosh']
            
            model.compile(
                loss=loss_fn,
                optimizer=optimizer,
                metrics=METRICS
            )

            # model.summary()
            _history = model.fit(self.x_train, self.y_train,
                              batch_size=self.batch_size,
                              epochs=int(budget),
                              verbose=0,
                              validation_data=(self.x_validation, self.y_validation))

            val_loss_per_epoch = _history.history['val_loss']
            best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1

            model.fit(self.x_train, self.y_train,
                              batch_size=self.batch_size,
                              epochs=best_epoch,
                              verbose=0,
                              validation_data=(self.x_validation, self.y_validation))

            train_score = model.evaluate(self.x_train, self.y_train, verbose=0)
            val_score = model.evaluate(self.x_validation, self.y_validation, verbose=0)
            test_score = model.evaluate(self.x_test, self.y_test, verbose=0)
            
            #print(test_score)

            resultsDF = pd.DataFrame([train_score,val_score,test_score],
                                     columns=the_columns,
                                     index=["train_score", "val_score", "test_score"],)
            # print(resultsDF)
            test_predictions_baseline = model.predict(self.x_test)
            np.savetxt(os.path.join(self.save_dic,'testing_finalResults_true.out'), self.y_test, delimiter=',')
            np.savetxt(os.path.join(self.save_dic,'testing_finalResults_pred.out'), test_predictions_baseline, delimiter=',')

            return ({
                'loss': test_score[0],  
                'info':  resultsDF.to_dict('index')
            })


    @staticmethod
    def get_configspace():
            """
            It builds the configuration space with the needed hyperparameters.
            It is easily possible to implement different types of hyperparameters.
            Beside float-hyperparameters on a log scale, it is also able to handle categorical input parameter.
            :return: ConfigurationsSpace-Object
            """
            cs = CS.ConfigurationSpace()

            lr = CSH.UniformFloatHyperparameter('lr', lower=1e-6, upper=1e-1, default_value='1e-2', log=True)

            # For demonstration purposes, we add different optimizers as categorical hyperparameters.
            # To show how to use conditional hyperparameters with ConfigSpace, we'll add the optimizers 'Adam' and 'SGD'.
            # SGD has a different parameter 'momentum'.
            optimizer = CSH.CategoricalHyperparameter('optimizer', ['Adam', 'SGD'])

            sgd_momentum = CSH.UniformFloatHyperparameter('sgd_momentum', lower=0.0, upper=0.99, default_value=0.9, log=False)

            cs.add_hyperparameters([lr, optimizer, sgd_momentum])



            num_dense_layers =  CSH.UniformIntegerHyperparameter('num_dense_layers', lower=1, upper=3, default_value=2)

            start_neurons_units = CSH.UniformIntegerHyperparameter('start_neurons_units', lower=32, upper=512, default_value=32, log=True)
            dense1_units = CSH.UniformIntegerHyperparameter('dense1_units', lower=8, upper=128, default_value=16, log=True)
            dense2_units = CSH.UniformIntegerHyperparameter('dense2_units', lower=4, upper=64, default_value=8, log=True)

            cs.add_hyperparameters([num_dense_layers, start_neurons_units, dense1_units, dense2_units])

            # start_neurons_activation = CSH.CategoricalHyperparameter('start_neurons_activation', ['relu', 'tanh', 'sigmoid'])
            # dense1_activation = CSH.CategoricalHyperparameter('dense1_activation', ['relu', 'tanh', 'sigmoid'])
            # dense2_activation = CSH.CategoricalHyperparameter('dense2_activation', ['relu', 'tanh', 'sigmoid'])
            # start_neurons_activation = CSH.CategoricalHyperparameter('start_neurons_activation', ['relu'])
            # dense1_activation = CSH.CategoricalHyperparameter('dense1_activation', ['relu'])
            # dense2_activation = CSH.CategoricalHyperparameter('dense2_activation', ['relu'])
            #
            # cs.add_hyperparameters([start_neurons_activation, dense1_activation, dense2_activation])

            dropout1_rate = CSH.UniformFloatHyperparameter('dropout1_rate', lower=0.0, upper=0.9, default_value=0.5, log=False)
            dropout2_rate = CSH.UniformFloatHyperparameter('dropout2_rate', lower=0.0, upper=0.9, default_value=0.5, log=False)

            cs.add_hyperparameters([dropout1_rate, dropout2_rate])


            # The hyperparameter sgd_momentum will be used,if the configuration
            # contains 'SGD' as optimizer.
            cond = CS.EqualsCondition(sgd_momentum, optimizer, 'SGD')
            cs.add_condition(cond)

            # You can also use inequality conditions:
            cond = CS.GreaterThanCondition(dense1_units, num_dense_layers, 1)
            cs.add_condition(cond)

            cond = CS.GreaterThanCondition(dense2_units, num_dense_layers, 2)
            cs.add_condition(cond)

            return cs


### help functions 


In [13]:
def load_sub_dataset_twostream(familiesSet, label_folder, feature_folder):
    # append all rows of subjects, and their lables
    allFrames = np.array([])
    allLables = np.array([])
    
    rgb_feature_folder = feature_folder.replace(featureType,'rgb')
    flow_feature_folder = feature_folder.replace(featureType,'flow')
    for this_family in familiesSet:
        onlyfiles = [f for f in os.listdir(rgb_feature_folder) if
                       os.path.isfile(os.path.join(rgb_feature_folder, f))
                       and f.startswith(this_family + '_')]
        onlyfiles.sort()

        for this_file in onlyfiles:
            this_lable_file = this_file.replace('_'+this_file.split('_')[4],'')+'.npy'
            currLabel = np.load(os.path.join(label_folder,this_lable_file))
            
            rgb_currData = np.load(os.path.join(rgb_feature_folder,this_file))    
            flow_currData = np.load(os.path.join(flow_feature_folder,this_file.replace('rgb','flow')))
            
            currData = np.hstack((rgb_currData, flow_currData))
            
            if allFrames.shape[0] ==0:
                allFrames = currData
                allLables = currLabel
            else:
                allFrames = np.vstack((allFrames, currData))
                allLables = np.hstack((allLables, currLabel))

    return allFrames, allLables

def load_sub_dataset(familiesSet, label_folder, feature_folder, featureType):
    # append all rows of subjects, and their lables
    allFrames = np.array([])
    allLables = np.array([])
    
    if featureType == 'twostream':
        return load_sub_dataset_twostream(familiesSet, label_folder, feature_folder)
    
    for this_family in familiesSet:
        # F10_Interaction_1_P27_rgb.npy
        onlyfiles = [f for f in os.listdir(feature_folder) if
                       os.path.isfile(os.path.join(feature_folder, f))
                       and f.startswith(this_family + '_')]
        onlyfiles.sort()

        for this_file in onlyfiles:
            currData = np.load(os.path.join(feature_folder,this_file))
            substr = '_'.join(this_file.split('_')[4:])
            this_lable_file = this_file.replace('_'+substr,'')+'.npy'
            currLabel = np.load(os.path.join(label_folder,this_lable_file))
            cutoff = min(len(currData), len(currLabel))
            currData = currData[:cutoff,:]
            currLabel =currLabel[:cutoff]
            
            if allFrames.shape[0] ==0:
                allFrames = currData
                allLables = currLabel
            else:
                allFrames = np.vstack((allFrames, currData))
                allLables = np.hstack((allLables, currLabel))

    return allFrames, allLables

def regression_random_oversampling(dataX, datay):
    low_channel_percentile = 10
    high_channel_percentile = 90
    relevance_threshold_percentile = 95 
       
    relevance = resreg.sigmoid_relevance(datay, cl=np.percentile(datay,low_channel_percentile),\
                                         ch=np.percentile(datay,high_channel_percentile))
    dataX, datay = resreg.random_oversample(dataX ,datay, relevance, 
                                relevance_threshold=np.percentile(relevance,relevance_threshold_percentile),\
                                over='balance', random_state=0)
    return dataX, datay


def load_dataset_selectedSubj(trainSubjs, valSubjs, testSubjs, label_folder, feature_folder,\
                              prblemType, featureType, num_classes):
    #simple sampeling method for refression
    #TODO: advanced ones SMOTER? <-- does it work for deep features 
    
    # load all train
    trainX, trainy = load_sub_dataset(trainSubjs, label_folder, feature_folder, featureType)
    trainX, trainy = shuffle(trainX, trainy)
#     _ = plt.hist(trainy, bins='auto')
#     plt.show()
    
    
    # Resample training data (Random Oversampling)   
    trainX, trainy = regression_random_oversampling(trainX, trainy)
#     _ = plt.hist(trainy, bins='auto')
#     plt.show()
 
    # load validation
    valX, valy = load_sub_dataset(valSubjs, label_folder, feature_folder, featureType)
    valX, valy = shuffle(valX, valy)
#     _ = plt.hist(valy, bins='auto')
#     plt.show()
    
    # Resample training data (Random Oversampling)
    valX, valy = regression_random_oversampling(valX ,valy)
#     _ = plt.hist(valy, bins='auto')
#     plt.show()
    
    
    # load all test
    testX, testy = load_sub_dataset(testSubjs, label_folder, feature_folder, featureType)
#     _ = plt.hist(testy, bins='auto')
#     plt.show()
    
    # Resample training data (Random Oversampling)
    testX, testy = regression_random_oversampling(testX ,testy)
#     _ = plt.hist(testy, bins='auto')
#     plt.show()

    return trainX, trainy, valX, valy , testX, testy, num_classes
    
def create_modlling(label_folder,feature_folder,result_folder, prblemType, featureType, num_classes):    
    # repeat experiment
    temp = {}
    all_trainSubjs = [['F' + str(i) for i in [1, 2, 3, 4, 5, 6, 8]]]
    all_valSubjs = [['F' + str(i) for i in [11, 17]]]
    all_testSubjs = [['F' + str(i) for i in [7, 10, 13]]]
    
    min_budget = 9
    max_budget = 243
    n_iterations = 50
    num_workers = 12

    for r in range(len(all_trainSubjs)):
        shared_directory = result_folder + '_'+ str(r) 
        if os.path.exists(shared_directory):
            print(shared_directory,' already processed')
            continue
        
        print(shared_directory,' under processing')
        classType = os.path.basename(shared_directory)

        # load data
        trainSubjs = all_trainSubjs[r]
        valSubjs = all_valSubjs[r]
        testSubjs = all_testSubjs[r]
        
        trainX, trainy, valX, valy, testX, testy, num_classes = \
            load_dataset_selectedSubj(trainSubjs, valSubjs, testSubjs, \
                                      label_folder, feature_folder, prblemType, featureType, num_classes)
        
        
        n_timesteps, n_features, n_outputs = trainX.shape[0], trainX.shape[1], num_classes        
        
        host = hpns.nic_name_to_host('lo')
        result_logger = hpres.json_result_logger(directory=shared_directory, overwrite=True)
        NS = hpns.NameServer(run_id=classType, host=host, port=0, working_directory=shared_directory)
        ns_host, ns_port = NS.start()
    
        workers = []
        for i in range(num_workers):
            worker = KerasWorker(n_features, n_outputs, prblemType, \
                                 trainX, trainy, valX, valy, testX, testy, \
                                 shared_directory,
                                 run_id=classType,host=host, nameserver=ns_host, nameserver_port=ns_port,
                                 id=i)
            worker.run(background=True)
            workers.append(worker)

        bohb = BOHB(configspace=worker.get_configspace(),
                  run_id=classType,
                  host=host,
                  nameserver=ns_host,
                  nameserver_port=ns_port,
                  result_logger=result_logger,
                  min_budget=min_budget, max_budget=max_budget
                    )
        res = bohb.run(n_iterations=1,  min_n_workers=num_workers)

        id2config = res.get_id2config_mapping()
        incumbent = res.get_incumbent_id()

        print('Best found configuration:', id2config[incumbent]['config'])
        # print('A total of %i unique configurations where sampled.' % len(id2config.keys()))
        # print('A total of %i runs where executed.' % len(res.get_all_runs()))
        # print('Total budget corresponds to %.1f full function evaluations.' % (
        #             sum([r.budget for r in res.get_all_runs()]) / max_budget))

        # store results
        with open(os.path.join(shared_directory, 'results.pkl'), 'wb') as fh:
            pickle.dump(res, fh)

        # shutdown
        bohb.shutdown(shutdown_workers=True)
        NS.shutdown()


### modeling

In [4]:
prblemTypes = ['regression']
featureTypes = ['rgb','flow', 'twostream']
classes = [9,5,3]
# divides = [2.5, 5]
fusionTypes = ['conc','avg','none']

permutations=[ prblemTypes, featureTypes, classes, fusionTypes]
all_permutations = list(itertools.product(*permutations))
print(len(all_permutations))
for this_permutation in all_permutations:
    (prblemType, featureType, eng_lvls, fusionType) = this_permutation
    classType = 'round_avg_eng_level' if prblemType == 'classification' else 'avg_eng_level'
    divide = 2.5 if fusionType == 'none' else 5
    
    print('Working on: ',prblemType, featureType, eng_lvls, divide, fusionType)
    
    label_folder = os.path.join(lables_path,'_'.join([classType,'eng_lvl',prblemType,str(eng_lvls),str(divide)]))

    
    if divide == 5:
        extra_txt = '_'.join([fusionType,str(divide)]) 
        feature_folder = os.path.join(features_path,'_'.join(['i3d',featureType,'features',extra_txt]))
    else:
        feature_folder = os.path.join(features_path,'_'.join(['i3d',featureType,'features']))
        
    
    #save results like: rgb_classification_9_2.5_none
    result_folder = os.path.join(results_path,'_'.join([featureType,prblemType,str(eng_lvls),str(divide),fusionType]))

    create_modlling(label_folder,feature_folder,result_folder, prblemType, featureType, eng_lvls)#, divide,fusionType)
    

27
Working on:  regression rgb 9 5 conc
./modelling_results/rgb_regression_9_5_conc_0  under processing


2021-10-28 18:08:05.020575: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-28 18:08:05.048305: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sharifa/catkin_ws/devel/lib:/usr/local/cuda-11.0/lib64
2021-10-28 18:08:05.050392: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sharifa/catkin_ws/devel/lib:/usr/local/cuda-11.0/lib64
2021-10-28 18:08:05.050406: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned ab

Best found configuration: {'dropout1_rate': 0.7287920973258931, 'dropout2_rate': 0.10169571820335176, 'lr': 0.0013457546042931275, 'num_dense_layers': 2, 'optimizer': 'Adam', 'start_neurons_units': 82, 'dense1_units': 71}
Working on:  regression rgb 9 5 avg
./modelling_results/rgb_regression_9_5_avg_0  under processing
Best found configuration: {'dropout1_rate': 0.48646939605159456, 'dropout2_rate': 0.589148418545302, 'lr': 0.000989323144392342, 'num_dense_layers': 2, 'optimizer': 'Adam', 'start_neurons_units': 34, 'dense1_units': 125}
Working on:  regression rgb 9 2.5 none
./modelling_results/rgb_regression_9_2.5_none_0  under processing
Best found configuration: {'dropout1_rate': 0.4421659173280363, 'dropout2_rate': 0.7767571774296114, 'lr': 2.4617823583142346e-05, 'num_dense_layers': 2, 'optimizer': 'Adam', 'start_neurons_units': 481, 'dense1_units': 95}
Working on:  regression rgb 5 5 conc
./modelling_results/rgb_regression_5_5_conc_0  under processing
Best found configuration: {'d

Best found configuration: {'dropout1_rate': 0.354374452638158, 'dropout2_rate': 0.8828550716490757, 'lr': 0.05355249552914864, 'num_dense_layers': 3, 'optimizer': 'SGD', 'start_neurons_units': 55, 'dense1_units': 126, 'dense2_units': 13, 'sgd_momentum': 0.4173627396248238}
Working on:  regression flow 3 5 conc
./modelling_results/flow_regression_3_5_conc_0  under processing
Best found configuration: {'dropout1_rate': 0.8181061805775379, 'dropout2_rate': 0.8010675370169473, 'lr': 0.0003030256397544131, 'num_dense_layers': 2, 'optimizer': 'Adam', 'start_neurons_units': 393, 'dense1_units': 67}
Working on:  regression flow 3 5 avg
./modelling_results/flow_regression_3_5_avg_0  under processing
Best found configuration: {'dropout1_rate': 0.8201169125459854, 'dropout2_rate': 0.30858676613825786, 'lr': 0.0003068900660178044, 'num_dense_layers': 2, 'optimizer': 'Adam', 'start_neurons_units': 76, 'dense1_units': 89}
Working on:  regression flow 3 2.5 none
./modelling_results/flow_regression_3_

### modelling for handcrafted 

In [ ]:
prblemTypes = ['regression']
featureTypes = ['handcrafted']
classes = [9,5,3]
# divides = [2.5, 5]
fusionTypes = ['2.5','5','5s']

permutations=[ prblemTypes, featureTypes, classes, fusionTypes]
all_permutations = list(itertools.product(*permutations))
print(len(all_permutations))
for this_permutation in all_permutations:
    (prblemType, featureType, eng_lvls, fusionType) = this_permutation
    classType = 'round_avg_eng_level' if prblemType == 'classification' else 'avg_eng_level'
    
    print('Working on: ',prblemType, featureType, eng_lvls, fusionType)
    
    label_folder = os.path.join(lables_path,'_'.join([classType,'eng_lvl',prblemType,str(eng_lvls),str(fusionType)]))
    feature_folder = os.path.join(features_path,'_'.join([featureType,'features',fusionType]))
        
    #save results like: rgb_classification_9_2.5_none
    result_folder = os.path.join(results_path,'_'.join([featureType,prblemType,str(eng_lvls),fusionType]))

    create_modlling(label_folder,feature_folder,result_folder, prblemType, featureType, eng_lvls)#, divide,fusionType)
   

9
Working on:  regression handcrafted 9 2.5
./modelling_results/handcrafted_regression_9_2.5_0  under processing


2021-11-03 00:31:49.815459: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-03 00:31:49.825765: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sharifa/catkin_ws/devel/lib:/usr/local/cuda-11.0/lib64
2021-11-03 00:31:49.826875: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sharifa/catkin_ws/devel/lib:/usr/local/cuda-11.0/lib64
2021-11-03 00:31:49.826896: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned ab

Best found configuration: {'dropout1_rate': 0.06928502265657847, 'dropout2_rate': 0.242897575655007, 'lr': 1.3705353174132053e-06, 'num_dense_layers': 2, 'optimizer': 'SGD', 'start_neurons_units': 61, 'dense1_units': 24, 'sgd_momentum': 0.7006122059958867}
Working on:  regression handcrafted 9 5
./modelling_results/handcrafted_regression_9_5_0  under processing
Best found configuration: {'dropout1_rate': 0.46257912355785086, 'dropout2_rate': 0.33783306560160464, 'lr': 0.005038575427971224, 'num_dense_layers': 3, 'optimizer': 'Adam', 'start_neurons_units': 44, 'dense1_units': 126, 'dense2_units': 9}
Working on:  regression handcrafted 9 5s
./modelling_results/handcrafted_regression_9_5s_0  under processing
